In [1]:
import torch
import torch.nn as nn
from torchtext import data
from torchtext import datasets
from torchtext import datasets
import random
import time
import torch.optim as optim

In [2]:
SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [3]:
TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [4]:
print(vars(train_data.examples[0]))

{'text': ['This', 'movie', ',', 'supposedly', 'a', 'thriller', ',', 'had', 'about', 'five', 'sub', '-', 'plots', 'that', 'developed', 'simultaneously', 'to', 'climax', 'at', 'the', 'end', ',', 'but', 'it', 'ended', 'up', 'more', 'as', 'a', 'yawn.<br', '/><br', '/>The', 'writing', 'was', 'trite', ',', 'the', 'pace', 'was', 'slow', 'and', 'disjointed', 'and', 'the', 'characters', 'were', 'boring', '.', 'George', 'Clooney', 'looks', 'like', 'he', 'needs', 'to', 'get', 'into', 'a', 'gym', ',', 'Matt', 'Damon', 'hammed', 'it', 'up', 'and', 'the', 'others', 'seemed', 'to', 'read', 'their', 'lines', 'from', 'a', 'teleprompter.<br', '/><br', '/>The', 'worst', 'part', 'of', 'this', 'movie', 'was', 'that', 'it', 'was', 'carefully', 'crafted', 'to', 'be', 'politically', 'correct', 'and', 'so', 'it', 'ended', 'up', 'saying', 'nothing', 'at', 'all', 'about', 'big', 'business', ',', 'oil', 'and', 'the', 'Middle', '-', 'East', '.', 'I', 'have', 'seen', 'documentaries', 'provide', 'more', 'excitement'

In [5]:
for i in range(len(train_data)):
  vars(train_data.examples[i]).get('text').reverse()

In [6]:
print(vars(train_data.examples[0]))

{'text': ['.', 'do', 'to', 'else', 'nothing', 'absolutely', "'s", 'there', 'if', 'only', 'it', 'Watch', '.', 'excitement', 'more', 'provide', 'documentaries', 'seen', 'have', 'I', '.', 'East', '-', 'Middle', 'the', 'and', 'oil', ',', 'business', 'big', 'about', 'all', 'at', 'nothing', 'saying', 'up', 'ended', 'it', 'so', 'and', 'correct', 'politically', 'be', 'to', 'crafted', 'carefully', 'was', 'it', 'that', 'was', 'movie', 'this', 'of', 'part', 'worst', '/>The', '/><br', 'teleprompter.<br', 'a', 'from', 'lines', 'their', 'read', 'to', 'seemed', 'others', 'the', 'and', 'up', 'it', 'hammed', 'Damon', 'Matt', ',', 'gym', 'a', 'into', 'get', 'to', 'needs', 'he', 'like', 'looks', 'Clooney', 'George', '.', 'boring', 'were', 'characters', 'the', 'and', 'disjointed', 'and', 'slow', 'was', 'pace', 'the', ',', 'trite', 'was', 'writing', '/>The', '/><br', 'yawn.<br', 'a', 'as', 'more', 'up', 'ended', 'it', 'but', ',', 'end', 'the', 'at', 'climax', 'to', 'simultaneously', 'developed', 'that', 'p

In [7]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

In [8]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device)

In [11]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [12]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0

    model.train()
    
    for batch in iterator:
        optimizer.zero_grad()
        text, text_lengths = batch.text
        text_lengths = text_lengths.cpu()
        predictions = model(text, text_lengths).squeeze(1)
        loss = criterion(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [13]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            text, text_lengths = batch.text
            text_lengths = text_lengths.cpu()
            predictions = model(text, text_lengths).squeeze(1)
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [123]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        self.factor = 2 if bidirectional else 1
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        self.rnn = nn.ModuleList([
                                  nn.LSTM(embedding_dim, hidden_dim, bidirectional = bidirectional),
                                  nn.Dropout(dropout),
                                  nn.LSTM(hidden_dim * self.factor, hidden_dim, bidirectional = bidirectional),
                                #   nn.Dropout(dropout),
                                  nn.LSTM(hidden_dim * self.factor, hidden_dim, bidirectional = bidirectional),
        ])
        self.fc = nn.Linear(hidden_dim * self.factor, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, text, text_lengths):

        embedded = self.dropout(self.embedding(text))
                
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        for layer in self.rnn:
            if not isinstance(layer, torch.nn.modules.dropout.Dropout):
                packed_embedded, (hidden, cell) = layer(packed_embedded)
            else:
                packed_embedded, packed_lengths = nn.utils.rnn.pad_packed_sequence(packed_embedded)
                packed_embedded = nn.utils.rnn.pack_padded_sequence(layer(packed_embedded), packed_lengths)
 
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)) if self.factor == 2 else self.dropout(hidden[-1,:,:])
        return self.fc(hidden)

In [124]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 64
OUTPUT_DIM = 1
BIDIRECTIONAL = True
DROPOUT = 0.2
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX).to(device)

In [125]:
pretrained_embeddings = TEXT.vocab.vectors
print(pretrained_embeddings.shape)

torch.Size([25002, 100])


In [126]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.1830, -1.1160,  2.3601,  ..., -1.6465,  0.2431,  0.5666],
        [-0.2326, -1.2659, -1.1949,  ...,  0.9952, -0.0324, -0.1769],
        [-0.6184,  0.0806,  1.7381,  ...,  0.0445, -0.5604, -0.3227]],
       device='cuda:0')

In [127]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.1830, -1.1160,  2.3601,  ..., -1.6465,  0.2431,  0.5666],
        [-0.2326, -1.2659, -1.1949,  ...,  0.9952, -0.0324, -0.1769],
        [-0.6184,  0.0806,  1.7381,  ...,  0.0445, -0.5604, -0.3227]],
       device='cuda:0')


In [128]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,783,977 trainable parameters


In [129]:

optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()
criterion = criterion.to(device)


In [130]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs



In [131]:
N_EPOCHS = 15

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 28s
	Train Loss: 0.601 | Train Acc: 66.29%
	 Val. Loss: 0.479 |  Val. Acc: 77.82%
Epoch: 02 | Epoch Time: 0m 27s
	Train Loss: 0.539 | Train Acc: 72.73%
	 Val. Loss: 0.438 |  Val. Acc: 80.86%
Epoch: 03 | Epoch Time: 0m 27s
	Train Loss: 0.503 | Train Acc: 76.48%
	 Val. Loss: 0.604 |  Val. Acc: 67.99%
Epoch: 04 | Epoch Time: 0m 27s
	Train Loss: 0.413 | Train Acc: 81.31%
	 Val. Loss: 0.480 |  Val. Acc: 80.70%
Epoch: 05 | Epoch Time: 0m 27s
	Train Loss: 0.274 | Train Acc: 89.09%
	 Val. Loss: 0.420 |  Val. Acc: 84.59%
Epoch: 06 | Epoch Time: 0m 27s
	Train Loss: 0.246 | Train Acc: 90.64%
	 Val. Loss: 0.302 |  Val. Acc: 87.89%
Epoch: 07 | Epoch Time: 0m 27s
	Train Loss: 0.194 | Train Acc: 92.83%
	 Val. Loss: 0.305 |  Val. Acc: 87.69%
Epoch: 08 | Epoch Time: 0m 27s
	Train Loss: 0.151 | Train Acc: 94.59%
	 Val. Loss: 0.352 |  Val. Acc: 87.73%
Epoch: 09 | Epoch Time: 0m 27s
	Train Loss: 0.124 | Train Acc: 95.73%
	 Val. Loss: 0.337 |  Val. Acc: 88.37%
Epoch: 10 | Epoch T

In [132]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.335 | Test Acc: 86.64%


In [133]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

In [134]:
predict_sentiment(model, "This film is bad")

0.08215125650167465

In [135]:
predict_sentiment(model, "This film is great")

0.9392727017402649